
# AAI612: Deep Learning & its Applications


*Notebook 2.1: Learning the XOR Functions*

<a href="https://colab.research.google.com/github/harmanani/AAI612/blob/main/Week2/Notebook2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
The MIT License (MIT)

Copyright (c) 2021 NVIDIA

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

### XOR Gate

This lab illustrates how the learning algorithm for multilevel feedforward networks works in practice using a simple XOR problem.  We use a the following three-neuron network:

<img src="https://github.com/harmanani/AAI612/blob/main/Week2/images/xor_neural_network.png?raw=1" style="width: 400px;">

The above network shows neurons N0, N1, and N2 with the bias inputs omitted.  We do not also show the weights in the figure. We use tanh as an activation function for N0 and N1 and the logistic sigmoid function as an activation function for the output neuron N2, and we use MSE as the loss function.

The XOR gates are the basis of computation and are often called in "half-adders", the foundation of being able to add numbers together. Here's the truth table for XOR:

### XOR (Exclusive Or) Gate

<table>

<tr>
<th colspan="3">XOR gate truth table</th>
</tr>

<tr>
<th colspan="2">Input</th>
<th>Output</th>
</tr>

<tr>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>

<tr>
<td>0</td>
<td>1</td>
<td>1</td>
</tr>

<tr>
<td>1</td>
<td>0</td>
<td>1</td>
</tr>

<tr>
<td>1</td>
<td>1</td>
<td>0</td>
</tr>

</table>

### Init Code for XOR Learning Example

In [1]:
import numpy as np

# Training data for XOR function
x_train_xor = [np.array([1.0, -1.0, -1.0]),  # Input: [1, 0, 0]
               np.array([1.0, -1.0, 1.0]),   # Input: [1, 0, 1]
               np.array([1.0, 1.0, -1.0]),   # Input: [1, 1, 0]
               np.array([1.0, 1.0, 1.0])]    # Input: [1, 1, 1]
y_train_xor = [0.0, 1.0, 1.0, 0.0]  # Output for XOR function


### Variables Needed to Track State of Neurons

In [3]:
# Neuron weight initialization function
def neuron_w(input_count):
    weights = np.zeros(input_count+1)
    for i in range(1, (input_count+1)):
        weights[i] = np.random.uniform(-1.0, 1.0)
    return weights

# Initialize weights for each neuron
n_w = [neuron_w(2), neuron_w(2), neuron_w(2)]  # Weights for 3 neurons
n_y = [0, 0, 0]  # Output of each neuron
n_error = [0, 0, 0]  # Error of each neuron


### Helper Functions for Backpropagation:

In [4]:
# Forward pass function
def forward_pass(x):
    global n_y, n_w
    n_y[0] = np.tanh(np.dot(n_w[0], x))  # Neuron 0
    n_y[1] = np.tanh(np.dot(n_w[1], x))  # Neuron 1
    n2_inputs = np.array([1.0, n_y[0], n_y[1]])  # Bias
    z2 = np.dot(n_w[2], n2_inputs)
    n_y[2] = 1.0 / (1.0 + np.exp(-z2))  # Logistic function

# Backward pass function
def backward_pass(y_truth):
    global n_error, n_y, n_w
    error_prime = -(y_truth - n_y[2])  # Derivative of loss function
    derivative = n_y[2] * (1.0 - n_y[2])  # Logistic derivative
    n_error[2] = error_prime * derivative
    derivative = 1.0 - n_y[0]**2  # tanh derivative
    n_error[0] = n_w[2][1] * n_error[2] * derivative
    derivative = 1.0 - n_y[1]**2  # tanh derivative
    n_error[1] = n_w[2][2] * n_error[2] * derivative

# Weight adjustment function
def adjust_weights(x):
    global n_w, n_y, n_error
    LEARNING_RATE = 0.1
    n_w[0] -= (x * LEARNING_RATE * n_error[0])
    n_w[1] -= (x * LEARNING_RATE * n_error[1])
    n2_inputs = np.array([1.0, n_y[0], n_y[1]])  # Bias
    n_w[2] -= (n2_inputs * LEARNING_RATE * n_error[2])


### Training Loop to Learn the XOR Function with Backpropagation

In [6]:
# Training loop
index_list = [0, 1, 2, 3]  # For randomizing training order

all_correct = False
iterations = 0
while not all_correct and iterations < 10000:  # Limit iterations
    all_correct = True
    np.random.shuffle(index_list)  # Randomize order
    for i in index_list:  # Train on all examples
        forward_pass(x_train_xor[i])
        backward_pass(y_train_xor[i])
        adjust_weights(x_train_xor[i])

    for i in range(len(x_train_xor)):  # Check if converged
        forward_pass(x_train_xor[i])
        # Check if output matches expected value
        if (((y_train_xor[i] < 0.5) and (n_y[2] >= 0.5)) or ((y_train_xor[i] >= 0.5) and (n_y[2] < 0.5))):
            all_correct = False
    iterations += 1

# Displaying the final weights and number of iterations
print("Final Weights:", n_w)
print("Iterations:", iterations)


Final Weights: [array([ 1.49115431, -1.24173489, -1.47550377]), array([0.17240125, 0.53326428, 0.90115118]), array([-0.77883979,  1.70326004,  1.08808986])]
Iterations: 348
